In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import time

2023-05-14 05:11:54.268501: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 05:11:54.504347: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-14 05:11:54.559277: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import tensorflow_hub as hub 

In [3]:
%matplotlib inline

In [4]:
# SSD with Mobilenet v2

model_handle = 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2'
hub_model = hub.load(model_handle)

2023-05-14 05:11:57.258881: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-14 05:11:57.263144: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-14 05:11:57.263491: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-14 05:11:57.263827: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the ap

In [9]:
 
def get_cropped_images(image_path,hub_model):
    image = cv2.imread(image_path)
    
    if image is not None:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (320, 320))
        image = image[np.newaxis, ...]
        # image = tf.image.convert_image_dtype(image, tf.float32)[tf.newaxis, ...]
        start_time = time.time()
        result = hub_model(image)
        end_time = time.time()
        result = {key:value.numpy()[0] for key,value in result.items()}
        
        # filter results where detection_classes == 1
        human_idx = np.where(result['detection_classes'] == 1)

        human_results = {}
        human_results['detection_boxes'] = result['detection_boxes'][human_idx]
        human_results['detection_scores'] = result['detection_scores'][human_idx]
        
        cropped_results = []
        output_image = image.copy()
        for i in range(len(human_results['detection_boxes'])):
            box = human_results['detection_boxes'][i]
            box = [int(x * 320) for x in box]
            conf_score = human_results['detection_scores'][i]
            if conf_score > 0.5:
                # write to output folder as image
                cropped_img = image[0][box[0]:box[2], box[1]:box[3]]
                if (cropped_img.shape[0] > 10) and (cropped_img.shape[1] > 10):
                    cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_RGB2BGR)
                    cropped_results.append(cropped_img.copy())
                    
                    # draw bounding box
                    cv2.rectangle(output_image, (box[1], box[0]), (box[3], box[2]), (0, 255, 0), 2)
                    
        return cropped_results, output_image
                    

In [6]:

# load classification modle 

clf_model = tf.keras.models.load_model('model/binary_22')

In [11]:
def predict_image_class(image, model,thresh = 0.22):
    # Load the saved model

    
    img_array = tf.keras.preprocessing.image.img_to_array(image)
    # scale the image
    img_array = tf.image.resize(img_array, [32*2, 32])
    
    img_array = tf.expand_dims(img_array, 0)
    img_array /= 255.

    # Make predictions
    predictions = model(img_array)
    predicted_thresh = predictions[0][0].numpy()
    print(predicted_thresh)
    if predicted_thresh < thresh:
        return 'others'
    else:
        return 'zomato'


In [12]:
classified_results = []

cropped_results, output_image = get_cropped_images('data/test1.jpg',hub_model)

if len(cropped_results)>= 1:

    for image in cropped_results:
        classified_results.append(predict_image_class(image,clf_model))


0.2167659
0.2153042
0.19858296
0.2154769


In [13]:
classified_results

['others', 'others', 'others', 'others']